# Chapter 15: Monoid, Semigroup

In [1]:
import Data.Monoid

data Optional a = Nada
                | Only a
                deriving (Eq, Show)

instance Monoid a => Monoid (Optional a) where
    mempty = Only mempty
    (Only x) `mappend` (Only y) = Only (x `mappend` y)
    only @ (Only _) `mappend` _ = only
    _ `mappend` only @ (Only _) = only
    _ `mappend` _               = Nada

(Only $ Sum 3) <> (Only $ Sum 5)
(Only $ Sum 3) <> Nada
Nada <> (Only $ Sum 5)
(Only $ Sum 3) <> (Only $ Sum 5) <> Nada
Nada <> Nada

Only (Sum {getSum = 8})

Only (Sum {getSum = 3})

Only (Sum {getSum = 5})

Only (Sum {getSum = 8})

Nada

In [2]:
newtype First' a = First { getFirst' :: Optional a } deriving (Eq, Show)

instance Monoid (First' a) where
    mempty = First Nada
    only @ (First (Only a)) `mappend` _ = only
    _ `mappend` other = other
    
import Test.QuickCheck

monoidLeftIdentity :: (Eq m, Monoid m) => m -> Bool
monoidLeftIdentity a = (mempty <> a) == a

monoidRightIdentity :: (Eq m, Monoid m) => m -> Bool
monoidRightIdentity a = (a <> mempty) == a

monoidAssoc :: (Eq m, Monoid m) => m -> m -> m -> Bool
monoidAssoc a b c = (a <> (b <> c)) == ((a <> b) <> c)

instance Arbitrary a => Arbitrary (Optional a) where
    arbitrary = frequency [(3, fmap Only arbitrary), (1, return Nada)]

instance Arbitrary a => Arbitrary (First' a) where
    arbitrary = fmap First arbitrary

type M = First' Int

quickCheck (monoidAssoc :: M -> M -> M -> Bool)
quickCheck (monoidLeftIdentity :: M -> Bool)
quickCheck (monoidRightIdentity :: M -> Bool)

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

## Semigroup Exercises

In [3]:
data Two a b = Two a b deriving (Eq, Show)
import Data.Semigroup
import Data.Monoid hiding((<>))
instance (Semigroup a, Semigroup b) => Semigroup (Two a b) where
    (Two a1 b1) <> (Two a2 b2) = Two (a1 <> a2) (b1 <> b2)

Two (Sum 1) "asd" <> Two (Sum 3) "wert"

instance (Arbitrary a, Arbitrary b) => Arbitrary (Two a b) where
    arbitrary = Two <$> arbitrary <*> arbitrary

semigroupAssoc :: (Eq m, Semigroup m) => m -> m -> m -> Bool
semigroupAssoc a b c = (a <> (b <> c)) == ((a <> b) <> c)

type S = Two (Sum Int) String
quickCheck (semigroupAssoc :: S -> S -> S -> Bool)

Two (Sum {getSum = 4}) "asdwert"

+++ OK, passed 100 tests.

In [4]:
data Or a b = Fst a | Snd b deriving (Eq, Show)

instance Semigroup (Or a b) where
    snd @ (Snd _) <> _ = snd
    _ <> snd @ (Snd _) = snd
    or <> _            = or

instance (Arbitrary a, Arbitrary b) => Arbitrary (Or a b) where
    arbitrary = frequency[(1, Fst <$> arbitrary),(1, Snd <$> arbitrary)]

type S = Or Int String
quickCheck (semigroupAssoc :: S -> S -> S -> Bool)

+++ OK, passed 100 tests.

In [5]:
newtype Combine a b = Combine { unCombine :: (a -> b) }

instance Semigroup b => Semigroup (Combine a b) where
    (Combine f) <> (Combine g) = Combine (\x -> f x <> g x)

let f = Combine $ \n -> Sum (n + 1)
let g = Combine $ \n -> Sum (n - 1)

unCombine (f <> g) 1
unCombine (f <> f) 1
unCombine (g <> f) 1

--instance (Arbitrary (a -> b)) => Arbitrary (Combine a b) where
--    arbitrary = Combine <$> arbitrary

-- @TODO quickcheck tests

Sum {getSum = 2}

Sum {getSum = 4}

Sum {getSum = 2}

In [6]:
newtype Comp a = Comp { unComp :: a -> a }

instance Semigroup (Comp a) where
    (Comp f) <> (Comp g) = Comp (f . g)

let f = unComp $ Comp (+1) <> Comp (*5)
f 5

26

In [7]:
data Validation a b = Failure a | Success b deriving (Eq, Show)

instance Semigroup a => Semigroup (Validation a b) where
    (Failure a1)          <> (Failure a2)          = Failure (a1 <> a2)
    success @ (Success _) <> _                     = success
    _                     <> success @ (Success _) = success

Success 1 <> Failure "blah"
Failure "woot" <> Failure "blah"
Success 1 <> Success 2
Failure "woot" <> Success 2

instance (Arbitrary a, Arbitrary b) => Arbitrary (Validation a b) where
    arbitrary = frequency[(1, Failure <$> arbitrary),(1, Success <$> arbitrary)]

type S = Validation String Int
quickCheck (semigroupAssoc :: S -> S -> S -> Bool)

Success 1

Failure "wootblah"

Success 1

Success 2

+++ OK, passed 100 tests.

## Monoid exercises

In [8]:
instance (Monoid a, Monoid b) => Monoid (Two a b) where
    mempty = 
        Two mempty mempty
    (Two a1 b1) `mappend` (Two a2 b2) = 
        Two (a1 `mappend` a2) (b1 `mappend` b2) -- @TODO: How to avoid writing this again?

type M = Two (Sum Int) String
quickCheck (monoidAssoc :: M -> M -> M -> Bool)
quickCheck (monoidLeftIdentity :: M -> Bool)
quickCheck (monoidRightIdentity :: M -> Bool)

instance Monoid b => Monoid (Combine a b) where
    mempty = Combine (const mempty)
    (Combine f) `mappend` (Combine g) = Combine (\x -> f x `mappend` g x)

let f = Combine $ \n -> Sum (n + 1)
unCombine (mappend f mempty) 1

-- @TODO quickcheck tests

instance Monoid (Comp a) where
    mempty = Comp id
    (Comp f) `mappend` (Comp g) = Comp (f . g)
    
-- @TODO quickcheck tests

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

Sum {getSum = 2}

In [9]:
newtype Mem s a = Mem { runMem :: s -> (a,s) }

instance Monoid a => Monoid (Mem s a) where
    mempty = Mem (\s -> (mempty,s))
    (Mem mem1) `mappend` (Mem mem2) = Mem mem where
        mem s = let (a1, s1) = mem1 s
                    (a2, s2) = mem2 s1
                in (a1 `mappend` a2, s2)

f' = Mem $ \s -> ("hi", s + 1)
f'' = Mem $ \s -> (" hola", s * 5)

rmzero = runMem mempty 0
rmleft = runMem (f' `mappend` mempty) 0
rmright = runMem (mempty `mappend` f') 0

rmleft
rmright
(rmzero :: (String, Int))
rmleft == runMem f' 0
rmright == runMem f' 0

runMem (f' `mappend` f'') 42
runMem (f' `mappend` f'' `mappend` mempty) 42
runMem (f' `mappend` f'' `mappend` f'') 42


runMem (mempty `mappend` f'') 42
runMem (f'' `mappend` mempty) 42

-- @TODO quickcheck tests

Line 1: Redundant bracket
Found:
(rmzero :: (String, Int))
Why not:
rmzero :: (String, Int)

("hi",1)

("hi",1)

("",0)

True

...

("hi hola",215)

("hi hola",215)

("hi hola hola",1075)

(" hola",210)

(" hola",210)